In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Mounted at /content/gdrive


In [2]:
%cd '/content/gdrive/MyDrive/PASCAL/VOCdevkit/VOC2007/'
%ls

/content/gdrive/MyDrive/PASCAL/VOCdevkit/VOC2007
Annotations/  ImageSets/   seg_df.csv          SegmentationObject/
df.csv        JPEGImages/  SegmentationClass/


In [3]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

import random

import cv2

import pickle

전처리한 df [Object Detection.ipynb](https://github.com/hodurie/AI_Study/blob/master/Implementation/Datasets/Object%20Detection.ipynb)

In [4]:
df = pd.read_csv('df.csv')
df.head()

,file_name,file_type,object_length,object,xmin,ymin,xmax,ymax,pose,truncated,difficult
0,000012.jpg,train,1,car,156,97,351,270,Rear,0,0
1,000017.jpg,train,2,person,185,62,279,199,Left,0,0
2,000017.jpg,train,2,horse,90,78,403,336,Left,0,0
3,000023.jpg,train,6,bicycle,9,230,245,500,Unspecified,1,0
4,000023.jpg,train,6,bicycle,230,220,334,500,Frontal,1,0


PASCAL VOC 2007 directory
```
/VOCdevkit//VOC2007/
├── df.csv                        # Object Detection 전처리 csv
├── seg.csv                       # Segmentation 전처리 csv
├── samples.pickle                # 선정 된 file 이름 pickle
└── Finetune                      # positive_list, negative_list txt 폴더
     ├── train                    # Segmentation 전처리 csv
     │   ├── bbox                # bbox csv 파일
     │   ├── *_n.csv / *_p.csv   # negative / positive csv (iou 0.5 기준 나눔)
     │   ├── Annotations         # negative / positive csv (iou 0.3 기준 나눔)
     │   ├── positive            # score 0.6 이상 bndboxes
     │        └── *_n.csv / *_p.csv
     │   └── bndboxes            # 기존 bndboxes
     │        └── *_n.csv / *_p.csv
     └── validation
          └── Annotations


# 기존 PASCAL VOC 2007 directory 구조
/VOCdevkit//VOC2007/
├── Annotations           
│   └── *.xml        
├── ImageSets
│   ├── Layout
│   │   └── *.txt
│   ├── Main
│   │   └── *.txt
│   └── Segmentation
│        └── *.txt
├── JPEGImages
│   └── .jpg
├── SegmentationClass
│   └── *.png
└── SegmentationObject
     └── *.png

```


In [5]:
finetune_root_dir = './Finetune/'

if not os.path.exists(finetune_root_dir):
    os.mkdir(finetune_root_dir)

for name in ['train', 'validation']:
    dst_root_dir = os.path.join(finetune_root_dir, name)

    if not os.path.exists(dst_root_dir):
        os.mkdir(dst_root_dir)

    dst_annotation_dir = os.path.join(finetune_root_dir, name, 'Annotations')
    if not os.path.exists(dst_annotation_dir):
        os.mkdir(dst_annotation_dir)

특정 obj 만 추출해서 sample 만들기

In [6]:
def sample_split(df, obj='car'):
    # car dataset 사용
    cond = df['object'] == obj
    df = df[cond]

    samples = {}

    for name in ['train', 'validation']:
        cond = df['file_type'] == name
        sample = df.loc[cond, 'file_name'].unique()

        length = len(sample)
        
        indices = random.sample(range(length), int(length / 2))
        
        samples[name] = sample[indices]

    return samples

In [7]:
def IoU(pred_box, target_box):
    '''
    pred_box = [4] 
    target_box = [N, 4]
    '''
    # (xmax - xmin) * (ymax - ymin)

    if len(target_box.shape) == 1:
        target_box = target_box[np.newaxis, :]

    areaA = (target_box[:, 2] - target_box[:, 0]) * (target_box[:, 3] - target_box[:, 1])
    areaB = (pred_box[2] - pred_box[0]) * (pred_box[3] - pred_box[1])

    xA = np.maximum(pred_box[0], target_box[:, 0])
    yA = np.maximum(pred_box[1], target_box[:, 1])
    xB = np.minimum(pred_box[2], target_box[:, 2])
    yB = np.minimum(pred_box[3], target_box[:, 3])
    
    intersection = np.maximum(0.0, xB - xA) * np.maximum(0.0, yB - yA)
    
    scores = intersection / (areaA + areaB - intersection)

    return scores

In [8]:
def region_proposals(jpg, gs):
    global df
    path = os.path.join('./JPEGImages/', jpg)
    img = cv2.imread(path)

    gs.setBaseImage(img)
    gs.switchToSelectiveSearchQuality()

    rects = gs.process()
    # rects[N, 4]
    # [[xmin, ymin, xmax, ymax],
    # [xmin, ymin, xmax, ymax], ... ]

    cond = df['file_name'] == jpg
    cols = ['xmin', 'ymin', 'xmax', 'ymax']
    bndboxes = np.array(df.loc[cond, cols])

    maximum_bndbox_size = 0

    for bndbox in bndboxes:
        xmin, ymin, xmax, ymax = bndbox
        bndbox_size = (xmax - xmin) * (ymax - ymin)
        if bndbox_size > maximum_bndbox_size:
            maximum_bndbox_size = bndbox_size

    iou_list = []
    for rect in rects:
        scores = IoU(rect, bndboxes)
        iou_list.append(max(scores))
    
    return iou_list, rects, maximum_bndbox_size, bndboxes

In [9]:
def parse_annotation_jpg(jpg, gs):
    iou_list, rects, maximum_bndbox_size, bndboxes = region_proposals(jpg, gs)

    positive_list = []
    negative_list = []

    for i in range(len(iou_list)):
        xmin, ymin, xmax, ymax = rects[i]
        rect_size = (xmax - xmin) * (ymax - ymin)

        iou_score = iou_list[i]

        if iou_score >= 0.5:
            positive_list.append(rects[i])
        
        if 0 < iou_score < 0.5 and rect_size > (maximum_bndbox_size / 5.0):
            negative_list.append(rects[i])
    
    return positive_list, negative_list

In [10]:
gs = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

samples = sample_split(df)

for name in ['train', 'validation']:
    total_num_positive = 0
    total_num_negative = 0
    
    for sample in samples[name]:
        positive_list, negative_list = parse_annotation_jpg(sample, gs)
        total_num_positive += len(positive_list)
        total_num_negative += len(negative_list)

        finetune_path = os.path.join(finetune_root_dir, name)

        positive_list_path = os.path.join(finetune_path, sample.replace('.jpg', '_p.csv'))
        negative_list_path = os.path.join(finetune_path, sample.replace('.jpg', '_n.csv'))
        
        np.savetxt(positive_list_path, np.array(positive_list), fmt='%d', delimiter=' ')
        np.savetxt(negative_list_path, np.array(negative_list), fmt='%d', delimiter=' ')

    print('%s positive num: %d' % (name, total_num_positive))
    print('%s negative num: %d' % (name, total_num_negative))

/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:21: RuntimeWarning: invalid value encountered in true_divide


train positive num: 9658
train negative num: 67121
validation positive num: 7425
validation negative num: 54730


In [11]:
with open('samples.pickle','wb') as fw:
    pickle.dump(samples, fw)

In [12]:
with open('samples.pickle', 'rb') as fr:
    samples = pickle.load(fr)

In [13]:
samples

{'train': array(['006748.jpg', '008784.jpg', '003083.jpg', '001937.jpg',
        '003806.jpg', '006822.jpg', '002559.jpg', '000153.jpg',
        '003971.jpg', '003484.jpg', '003551.jpg', '004687.jpg',
        '005090.jpg', '002490.jpg', '008388.jpg', '002134.jpg',
        '008037.jpg', '002116.jpg', '005499.jpg', '008197.jpg',
        '000431.jpg', '001780.jpg', '009283.jpg', '008232.jpg',
        '007479.jpg', '003261.jpg', '001414.jpg', '006734.jpg',
        '000888.jpg', '003355.jpg', '008391.jpg', '008549.jpg',
        '001494.jpg', '004481.jpg', '006079.jpg', '008174.jpg',
        '003420.jpg', '005047.jpg', '009392.jpg', '009424.jpg',
        '000522.jpg', '001112.jpg', '006224.jpg', '000590.jpg',
        '007898.jpg', '004091.jpg', '000134.jpg', '002625.jpg',
        '003231.jpg', '008909.jpg', '005831.jpg', '003103.jpg',
        '000026.jpg', '008466.jpg', '007305.jpg', '001881.jpg',
        '004705.jpg', '006931.jpg', '002478.jpg', '001119.jpg',
        '009810.jpg', '000754.j

In [14]:
def parse_annotation_jpeg_svm(jpg, gs):
    iou_list, rects, maximum_bndbox_size, bndboxs = region_proposals(jpg, gs)

    positive_list = []
    negative_list = []

    for i in range(len(iou_list)):
        xmin, ymin, xmax, ymax = rects[i]
        rect_size = (ymax - ymin) * (xmax - xmin)

        iou_score = iou_list[i]

        if 0 < iou_score <= 0.3 and rect_size > maximum_bndbox_size / 5.0:
            negative_list.append(rects[i])
        
    return bndboxs, negative_list

In [ ]:
gs = cv2.ximgproc.segmentation.createSelectiveSearchSegmentation()

for name in ['train', 'validation']:
    total_num_positive = 0
    total_num_negative = 0

    for sample in samples[name]:
        positive_list, negative_list = parse_annotation_jpeg_svm(sample, gs)
        total_num_positive += len(positive_list)
        total_num_negative += len(negative_list)

        path = os.path.join(finetune_root_dir, name, 'Annotations')
        dst_annotation_positive_path = os.path.join(path, sample.replace('.jpg', '_p.csv'))
        dst_annotation_negative_path = os.path.join(path, sample.replace('.jpg', '_n.csv'))

        np.savetxt(dst_annotation_positive_path, np.array(positive_list), fmt='%d', delimiter=' ')
        np.savetxt(dst_annotation_negative_path, np.array(negative_list), fmt='%d', delimiter=' ')
    
    print('%s positive num: %d' % (name, total_num_positive))
    print('%s negative num: %d' % (name, total_num_negative))

In [ ]:
res_samples = []
total_positive_num = 0

name = 'train'

bbox_csv_path = os.path.join(finetune_root_dir, name, 'bbox')
bndboxes_path = os.path.join(finetune_root_dir, name, 'bndboxes')
positive_path = os.path.join(finetune_root_dir, name, 'positive')

if not os.path.exists(bndboxes_csv_path):
    os.mkdir(bndboxes_csv_path)

if not os.path.exists(positive_csv_path):
    os.mkdir(positive_csv_path)

for sample in samples[name]:
    # ex) 'Finetune/train/00001_p.csv
    path = os.path.join(finetune_root_dir, name, sample.replace('.jpg', '_p.csv'))
    # positive_bndboxes = [N, 4]
    positive_bndboxes = np.loadtxt(path, dtype=np.int, delimiter=' ')

    cols = ['xmin', 'ymin', 'xmax', 'ymax']
    train_df = df[df['file_name'] == sample]
    train_car = train_df.loc[df['object'] == 'car']

    # bndboxes = [N, 4]
    bndboxes = np.array(train_car[cols])
    
    positive_list = []

    # positive_bndboxes가 값을 하나 갖고 있을 때 (4)
    if len(positive_bndboxes.shape) == 1 and len(positive_bndboxes) != 0:
        scores = IoU(positive_bndboxes, bndboxes)
        if np.max(scores) > 0.6:
            positive_list.append(positive_bndboxes)
    # positive_bndboxes가 값을 여러 개 갖고 있을 때 (N, 4)
    elif len(positive_bndboxes.shape) == 2:
        for positive_bndbox in positive_bndboxes:
            scores = IoU(positive_bndbox, bndboxes)
            if np.max(scores) > 0.6:
                positive_list.append(positive_bndbox)

    if len(positive_list) > 0:
        # ex) 'Finetune/train/bndboxes/00001.csv
        bndboxes_paths = os.path.join(bndboxes_path, sample.replace('jpg', 'csv'))
        np.savetxt(bndboxes_paths, bndboxes, fmt='%s', delimiter=' ')

        # ex) 'Finetune/train/positive/00001.csv
        positive_csv_paths = os.path.join(positive_csv_path, sample.replace('jpg', 'csv'))
        np.savetxt(positive_csv_paths, np.array(positive_list), fmt='%s', delimiter=' ')

        total_positive_num += len(positive_list)
        res_samples.append(sample)
        print('save {} done'.format(sample))
    else:
        print('{} ineligible'.format(sample))

bnd_csv_path = os.path.join(bbox_csv_path, 'df.csv')
np.savetxt(bnd_csv_path, res_samples, fmt='%s', delimiter=' ')
print('total positive num: {}'.format(total_positive_num))

-------- 004544.jpg ineligible
-------- 000431.jpg ineligible
-------- 001902.jpg ineligible
save 005483.jpg done
save 008633.jpg done
save 006524.jpg done
save 006900.jpg done
save 002704.jpg done
save 009336.jpg done
save 009406.jpg done
save 009434.jpg done


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: loadtxt: Empty input file: "./Finetune/train/001902_p.csv"
  del sys.path[0]


-------- 008037.jpg ineligible
save 002544.jpg done
save 001334.jpg done
save 005047.jpg done
-------- 000296.jpg ineligible
-------- 006104.jpg ineligible
save 003007.jpg done
save 003103.jpg done
save 000083.jpg done
-------- 008676.jpg ineligible
-------- 004973.jpg ineligible
-------- 001060.jpg ineligible
-------- 004808.jpg ineligible
-------- 003835.jpg ineligible
save 008001.jpg done
-------- 007736.jpg ineligible


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: loadtxt: Empty input file: "./Finetune/train/004973_p.csv"
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: loadtxt: Empty input file: "./Finetune/train/001060_p.csv"
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: loadtxt: Empty input file: "./Finetune/train/003835_p.csv"
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: loadtxt: Empty input file: "./Finetune/train/007736_p.csv"
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: loadtxt: Empty input file: "./Finetune/train/002783_p.csv"
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: loadtxt: Empty input file: "./Finetune/train/005756_p.csv"
  del sys.path[0]


save 006369.jpg done
-------- 000906.jpg ineligible
-------- 007446.jpg ineligible
save 001881.jpg done
-------- 002783.jpg ineligible
-------- 005756.jpg ineligible
-------- 008482.jpg ineligible
save 001954.jpg done
-------- 006748.jpg ineligible
save 004873.jpg done
save 002311.jpg done
-------- 004830.jpg ineligible
save 001604.jpg done
total positive num: 829


/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: loadtxt: Empty input file: "./Finetune/train/008482_p.csv"
  del sys.path[0]
/usr/local/lib/python3.7/dist-packages/ipykernel_launcher.py:13: UserWarning: loadtxt: Empty input file: "./Finetune/train/006748_p.csv"
  del sys.path[0]


In [ ]:
import torch
import torch.nn as nn
import torch.nn.functional as F

import torch.utils.data as data
from torch.utils.data import DataLoader

In [ ]:
class CustomFinetuneDataset():
    def __init__(self, root_dir, transform=None):
        '''
        root_dir = 'Finetune/train'
        jpg_path = 'JPEGImages/'
        '''
        with open('samples.pickle', 'rb') as fr:
            samples = pickle.load(fr)

        name = root_dir.split('/')[-1]

        jpg_path = './JPEGImages/'
        
        images = [cv2.imread(os.path.join(jpg_path, sample)) for sample in samples[name]]

        annotations_path = os.path.join(root_dir, 'Annotations')
        annotations_dir = os.listdir(annotations_path)
    
        positive_annotations = [annotation for annotation in annotations_dir if annotation.endswith('_p.csv')]
        negative_annotations = [annotation for annotation in annotations_dir if annotation.endswith('_n.csv')]
        
        positive_sizes = []
        negative_sizes = []
        
        positive_rects = []
        negative_rects = []

        for annotation in positive_annotations:
            rects = np.loadtxt(os.path.join(annotations_path, annotation), dtype=np.int, delimiter=' ')

            if len(rects.shape) == 1:
                if rects.shape[0] == 4:
                    positive_rects.append(rects)
                    positive_sizes.append(1)
                else:
                    positive_sizes.append(0)
            else:
                positive_rects.extend(rects)
                positive_sizes.append(len(rects))
        
        for annotation in negative_annotations:
            rects = np.loadtxt(os.path.join(annotations_path, annotation), dtype=np.int, delimiter=' ')
            if len(rects.shape) == 1:
                if rects.shape[0] == 4:
                    negative_rects.append(rects)
                    negative_sizes.append(1)
                else:
                    positive_sizes.append(0)
            else:
                negative_rects.extend(rects)
                negative_sizes.append(len(rects))

        self.transform = transform
        self.images = images
        self.positive_sizes = positive_sizes
        self.negative_sizes = negative_sizes
        self.positive_rects = positive_rects
        self.negative_rects = negative_rects
        self.total_positive_num = int(np.sum(positive_sizes))
        self.total_negative_num = int(np.sum(negative_sizes))
    
    def __getitem__(self, index):
        image_id = len(self.images) - 1
        if index < self.total_positive_num:
            target = 1
            xmin, ymin, xmax, ymax = self.positive_rects[index]

            for i in range(len(self.positive_sizes) - 1):
                if np.sum(self.positive_sizes[:i]) <= index < np.sum(self.positive_sizes[:(i + 1)]):
                    image_id = i
                    break
            image = self.images[image_id][ymin:ymax, xmin:xmax]
        else:
            target = 0
            idx = index - self.total_positive_num
            xmin, ymin, xmax, ymax = self.negative_rects[idx]

            for i in range(len(self.negative_sizes) - 1):
                if np.sum(self.negative_sizes[:i]) <= idx < np.sum(self.negative_sizes[:(i + 1)]):
                    image_id = i
                    break
            image = self.jpeg_images[image_id][ymin:ymax, xmin:xmax]
        
        if self.transform:
            image = self.transform(image)

        return image, target

    def __len__(self):
        return self.total_positive_num + self.total_negative_num

    def get_positive_num(self):
        return self.total_positive_num

    def get_negative_num(self):
        return self.total_negative_num

In [ ]:
from google.colab.patches import cv2_imshow

def test_finetune(idx):
    root_dir = './Finetune/train'
    train_data_set = CustomFinetuneDataset(root_dir)

    print('positive num: %d' % train_data_set.get_positive_num())
    print('negative num: %d' % train_data_set.get_negative_num())
    print('total num: %d' % train_data_set.__len__())

    image, target = train_data_set.__getitem__(idx)
    print('target: %d' % target)

    cv2_imshow(image)
    cv2.waitKey(0)

In [ ]:
test_finetune(0)

In [ ]:
class CustomClassifierDataset(Dataset):
    def __init__(self, root_dir, transform=None):
        '''
        root_dir = 'Finetune/validation'
        jpg_path = 'JPEGImages/'
        '''
        with open('samples.pickle', 'rb') as fr:
            samples = pickle.load(fr)

        images = []
        positive_list = []
        negative_list = []

        name = root_dir.split('/')[-1]

        jpg_path = 'JPEGImages/'

        for idx in range(len(samples[name])):
            sample = samples[name][idx]
            images.append(cv2.imread(os.path.join(jpg_path, sample)))

            positive_annotation_path = os.path.join(root_dir, 'Annotations', sample.replace('.jpg', '_p.csv'))
            positive_annotations = np.loadtxt(positive_annotation_path, dtype=np.int, delimiter=' ')
    
            if len(positive_annotations.shape) == 1:
                if positive_annotations.shape[0] == 4:
                    positive_dict = dict()

                    positive_dict['rect'] = positive_annotations
                    positive_dict['image_id'] = idx
                    positive_list.append(positive_dict)
            else:
                for positive_annotation in positive_annotations:
                    positive_dict = dict()

                    positive_dict['rect'] = positive_annotation
                    positive_dict['image_id'] = idx
                    positive_list.append(positive_dict)

            negative_annotation_path = os.path.join(root_dir, 'Annotations', sample.replace('.jpg', '_p.csv'))
            negative_annotations = np.loadtxt(negative_annotation_path, dtype=np.int, delimiter=' ')

            if len(negative_annotations.shape) == 1:
                if negative_annotations.shape[0] == 4:
                    negative_dict = dict()
                    negative_dict['rect'] = negative_annotations
                    negative_dict['image_id'] = idx
                    negative_list.append(negative_dict)
            else:
                for negative_annotation in negative_annotations:
                    negative_dict = dict()

                    negative_dict['rect'] = negative_annotation
                    negative_dict['image_id'] = idx
                    negative_list.append(negative_dict)

        self.transform = transform
        self.images = images
        self.positive_list = positive_list
        self.negative_list = negative_list

    def __getitem__(self, index):
        if index < len(self.positive_list):
            target = 1
            positive_dict = self.positive_list[index]

            xmin, ymin, xmax, ymax = positive_dict['rect']
            image_id = positive_dict['image_id']

            image = self.images[image_id][ymin:ymax, xmin:xmax]
            cache_dict = positive_dict
        else:
            target = 0
            idx = index - len(self.positive_list)
            negative_dict = self.negative_list[idx]

            xmin, ymin, xmax, ymax = negative_dict['rect']
            image_id = negative_dict['image_id']

            image = self.images[image_id][ymin:ymax, xmin:xmax]
            cache_dict = negative_dict

        if self.transform:
            image = self.transform(image)

        return image, target, cache_dict

    def __len__(self):
        return len(self.positive_list) + len(self.negative_list)

    def get_transform(self):
        return self.transform

    def get_jpeg_images(self):
        return self.images

    def get_positive_num(self):
        return len(self.positive_list)

    def get_negative_num(self):
        return len(self.negative_list)

    def get_positives(self):
        return self.positive_list

    def get_negatives(self):
        return self.negative_list

    def set_negative_list(self, negative_list):
        self.negative_list = negative_list

In [ ]:
from google.colab.patches import cv2_imshow

def test_classifier(idx):
    root_dir = 'Finetune/validation'
    train_data_set = CustomClassifierDataset(root_dir)

    print('positive num: %d' % train_data_set.get_positive_num())
    print('negative num: %d' % train_data_set.get_negative_num())
    print('total num: %d' % train_data_set.__len__())

    image, target, cache_dict = train_data_set.__getitem__(idx)
    print('target: %d' % target)
    print('dict: ' + str(cache_dict))

    cv2_imshow(image)
    cv2.waitKey(0)

In [ ]:
test_classifier(0)

positive num: 92
negative num: 92
total num: 184
target: 1
dict: {'rect': array([324, 207, 385, 238]), 'image_id': 0}


In [ ]:
class BBoxRegressionDataset():
    def __init__(self, root_dir, transform=None):
        '''
        root_dir = 'Finetune/train'
        
        bndboxes path = 'Finetune/train/bndboxes'
        positive path = 'Finetune/train/positive'
        두 파일과 관련 된 csv = 'Finetune/train/bbox'
        '''
        super(BBoxRegressionDataset, self).__init__()
        self.transform = transform

        name = root.split('/')[-1]
        txt = np.loadtxt(os.path.join(root_dir, 'bbox', 'df.csv'), dtype=np.str)

        jpg_path = 'JPEGImages/'

        images = []
        box_list = []

        for i in range(len(txt)):
            sample = txt[i]

            jpeg_path = os.path.join(jpg_path, sample)
            bndbox_path = os.path.join(root_dir, 'bndboxes', sample.replace('jpg', 'csv'))
            positive_path = os.path.join(root_dir, 'positive', sample.replace('jpg', 'csv'))

            images.append(cv2.imread(jpeg_path))
            bndboxes = np.loadtxt(bndbox_path, dtype=np.int, delimiter=' ')
            positives = np.loadtxt(positive_path, dtype=np.int, delimiter=' ')

            if len(positives.shape) == 1:
                bndbox = self.get_bndbox(bndboxes, positives)
                box_list.append({'image_id': i, 'positive': positives, 'bndbox': bndbox})
            else:
                for positive in positives:
                    bndbox = self.get_bndbox(bndboxes, positive)
                    box_list.append({'image_id': i, 'positive': positive, 'bndbox': bndbox})

        self.images = images
        self.box_list = box_list

    def __getitem__(self, index: int):
        assert index < self.__len__(), 'The data set size is %d, and the current input subscript is %d' % (self.__len__(), index)

        box_dict = self.box_list[index]
        image_id = box_dict['image_id']
        positive = box_dict['positive']
        bndbox = box_dict['bndbox']

        jpeg_img = self.images[image_id]
        xmin, ymin, xmax, ymax = positive
        image = jpeg_img[ymin:ymax, xmin:xmax]

        if self.transform:
            image = self.transform(image)

        target = dict()
        p_w = xmax - xmin
        p_h = ymax - ymin
        p_x = xmin + p_w / 2
        p_y = ymin + p_h / 2

        xmin, ymin, xmax, ymax = bndbox
        g_w = xmax - xmin
        g_h = ymax - ymin
        g_x = xmin + g_w / 2
        g_y = ymin + g_h / 2

        t_x = (g_x - p_x) / p_w
        t_y = (g_y - p_y) / p_h
        t_w = np.log(g_w / p_w)
        t_h = np.log(g_h / p_h)

        return image, np.array((t_x, t_y, t_w, t_h))

    def __len__(self):
        return len(self.box_list)

    def get_bndbox(self, bndboxes, positive):
        if len(bndboxes.shape) == 1:
            return bndboxes
        else:
            scores = IoU(positive, bndboxes)
            return bndboxes[np.argmax(scores)]

In [ ]:
import torchvision.transforms as transforms

def test_bbox():
    transform = transforms.Compose([
        transforms.ToPILImage(),
        transforms.Resize((227, 227)),
        transforms.RandomHorizontalFlip(),
        transforms.ToTensor(),
        transforms.Normalize((0.5, 0.5, 0.5), (0.5, 0.5, 0.5))
    ])

    root_dir = 'Finetune/train'
    data_set = BBoxRegressionDataset(root_dir, transform=transform)

    print(data_set.__len__())
    image, target = data_set.__getitem__(10)
    print(image.shape)
    print(target)
    print(target.dtype)

In [ ]:
test_bbox()

In [ ]:
from torch.utils.data import Sampler

class CustomBatchSampler(Sampler):
    def __init__(self, num_positive, num_negative, batch_positive, batch_negative):
        """
        2 classification data set
        Batch processing each time, including batch_positive positive samples and batch_negative negative samples
        @param num_positive: number of positive samples
        @param num_negative: number of negative samples
        @param batch_positive: the number of positive samples at a time
        @param batch_negative: number of negative samples at a time
        """
        self.num_positive = num_positive
        self.num_negative = num_negative
        self.batch_positive = batch_positive
        self.batch_negative = batch_negative

        length = num_positive + num_negative
        self.idx_list = list(range(length))

        self.batch = batch_negative + batch_positive
        self.num_iter = length // self.batch

    def __iter__(self):
        sampler_list = list()
        for i in range(self.num_iter):
            tmp = np.concatenate(
                (random.sample(self.idx_list[:self.num_positive], self.batch_positive),
                 random.sample(self.idx_list[self.num_positive:], self.batch_negative))
            )
            random.shuffle(tmp)
            sampler_list.extend(tmp)
        return iter(sampler_list)

    def __len__(self) -> int:
        return self.num_iter * self.batch

    def get_num_batch(self) -> int:
        return self.num_iter


## Reference
- [R-CNN](https://github.com/object-detection-algorithm/R-CNN)

```
@misc{girshick2013rich,
    title={Rich feature hierarchies for accurate object detection and semantic segmentation},
    author={Ross Girshick and Jeff Donahue and Trevor Darrell and Jitendra Malik},
    year={2013},
    eprint={1311.2524},
    archivePrefix={arXiv},
    primaryClass={cs.CV}
}

@misc{pascal-voc-2007,
	author = "Everingham, M. and Van~Gool, L. and Williams, C. K. I. and Winn, J. and Zisserman, A.",
	title = "The {PASCAL} {V}isual {O}bject {C}lasses {C}hallenge 2007 {(VOC2007)} {R}esults",
	howpublished = "http://www.pascal-network.org/challenges/VOC/voc2007/workshop/index.html"}

```